# LangChain Study and Practice

In [23]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_key = os.environ.get("OPENAI_API_KEY")
serp_key = os.environ.get("SERP_API_KEY")
gplaces_key = os.environ.get("GPLACES_API_KEY")
openweathermap_key = os.environ.get("OPENWEATHERMAP_API_KEY")

## Generic

In [10]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

### One Input Variable

In [11]:
# Prompt Generation

prompt = PromptTemplate(
    input_variables = ["product"], # this would be the user input
    template = "What is a good name for a company that makes {product}"
)

print(prompt.format(product = "podcast player")) # for the example input, the product would be "podcast player"

What is a good name for a company that makes podcast player


In [12]:
# OpenAI API

chatopenai = ChatOpenAI(model_name = "gpt-3.5-turbo")
llmchain_chat = LLMChain(llm = chatopenai, prompt = prompt)
llmchain_chat.run("podcast player")

'PodcastPulse'

### Multiple Input Variables

In [13]:
# Prompt Generation

prompt = PromptTemplate(
    input_variables = ["product", "audience"],
    template = "What is a good name for a company that makes {product} for {audience}"
)

print(prompt.format(product = "podcast player", audience = "children"))

What is a good name for a company that makes podcast player for children


In [14]:
# OpenAI API

llmchain_chat.run({"product": "podcast player", "audience": "children"}) # Multiple variables are required to be passed as a dict

'PodcastStream'

## Utility

In [15]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

### Test Google Places API

In [16]:
from langchain.tools import GooglePlacesTool

In [17]:
places = GooglePlacesTool()

In [18]:
places.run("al fronos")

'1. Al Forno PH\nAddress: 1, 9A Villa Arca Ave, Quezon City, Metro Manila, Philippines\nGoogle place ID: ChIJceWfvmu3lzMRCVhnuddMf3A\nPhone: Unknown\nWebsite: Unknown\n\n'

### Serp API

In [24]:
# Tools and Initializing Agent

llm = OpenAI(temperature = 0)

tool_names = ["serpapi"]
tools = load_tools(tool_names)

agent = initialize_agent(tools, llm, agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose = True)

In [41]:
query = input("Question: ")

In [42]:
query

'What are some good vacation spots in South Luzon Philippines?'

In [43]:
answer = agent.run(query)



> Entering new AgentExecutor chain...
 I should research some popular vacation spots in South Luzon
Action: Search
Action Input: "Vacation spots in South Luzon Philippines"
Observation: 10 Top Destinations in Southern Luzon, Philippines · 10. Legazpi · 9. Catanduanes · 8. Caramoan Peninsula · 7. Calaguas Islands · 6. Mabini · 5.
Thought: I should look at the individual destinations to get more information
Action: Search
Action Input: "Legazpi Philippines"
Observation: Legazpi, officially the City of Legazpi, is a 1st class component city and capital of the province of Albay, Philippines. According to the 2020 census, it has a population of 209,533. Legazpi is the regional center and largest city of the Bicol Region, in terms of population.
Thought: I should look at more destinations
Action: Search
Action Input: "Catanduanes Philippines"
Observation: Catanduanes, officially the Province of Catanduanes, is an island province located in the Bicol Region of Luzon in the Philippines. It i

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-8s0GpBTuKfLf5Ae3c3Bdn0jI on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-8s0GpBTuKfLf5Ae3c3Bdn0jI on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

 I now know the popular vacation spots in South Luzon
Final Answer: Popular vacation spots in South Luzon, Philippines include Legazpi, Catanduanes, Caramoan Peninsula, Calaguas Islands, Mabini, and more.

> Finished chain.


In [44]:
answer

'Popular vacation spots in South Luzon, Philippines include Legazpi, Catanduanes, Caramoan Peninsula, Calaguas Islands, Mabini, and more.'

### Extraction Chain

In [45]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain

# Schema
schema = {
    "properties": {
        "location": {"type": "string"},
    },
    "required": ["location"],
}

llm = ChatOpenAI(temperature = 0, model = "gpt-3.5-turbo")
chain = create_extraction_chain(schema, llm)
locations = chain.run(answer)

In [46]:
locations

[{'location': 'Legazpi'},
 {'location': 'Catanduanes'},
 {'location': 'Caramoan Peninsula'},
 {'location': 'Calaguas Islands'},
 {'location': 'Mabini'}]

In [47]:
for item in locations:
    location = item['location']
    print(location)

Legazpi
Catanduanes
Caramoan Peninsula
Calaguas Islands
Mabini


### Google Places API

In [48]:
results = []

for item in locations:
    location = item['location']
    result = places.run(location) 
    results.append(result)

In [49]:
results

['1. Legazpi City\nAddress: Legazpi City, Albay, Philippines\nGoogle place ID: ChIJp_ibfmgBoTMRelMUjedSUjA\nPhone: Unknown\nWebsite: http://www.legazpicity.gov.ph/\n\n',
 '1. Catanduanes\nAddress: Catanduanes, Philippines\nGoogle place ID: ChIJy9J5j-v1nzMRVLLVwlWbUE0\nPhone: Unknown\nWebsite: Unknown\n\n',
 '1. Caramoan Peninsula\nAddress: Caramoan Peninsula, Camarines Sur, Philippines\nGoogle place ID: ChIJ5SBwqKvWoTMRKk5FXmOVYA0\nPhone: Unknown\nWebsite: Unknown\n\n',
 '1. Calaguas\nAddress: Calaguas, Vinzons, Camarines Norte, Philippines\nGoogle place ID: ChIJawZK6ZnPmDMRVlVIBSkBSPo\nPhone: Unknown\nWebsite: Unknown\n\n',
 '1. Mabini\nAddress: Mabini, Batangas, Philippines\nGoogle place ID: ChIJLbgdokcHvTMRRITNRyJ3h2k\nPhone: Unknown\nWebsite: Unknown\n\n']

### OpenWeather API

In [50]:
from langchain.utilities import OpenWeatherMapAPIWrapper

In [51]:
weather = OpenWeatherMapAPIWrapper()

In [52]:
weather_data = weather.run("Manila,PH")
print(weather_data)

In Manila,PH, the current weather is as follows:
Detailed status: light rain
Wind speed: 2.57 m/s, direction: 240°
Humidity: 88%
Temperature: 
  - Current: 28.11°C
  - High: 28.96°C
  - Low: 26.66°C
  - Feels like: 33.74°C
Rain: {'1h': 0.19}
Heat index: None
Cloud cover: 20%


### Multiple Tools

In [30]:
tool_names = ["serpapi", "openweathermap-api"]
tools = load_tools(tool_names)

agent = initialize_agent(tools, llm, agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose = True)

In [31]:
# Input for weather API

query = ""

query = input("Question")

query

'What is the weather like right now in London UK?'

In [32]:
agent.run(query)



> Entering new AgentExecutor chain...
I need to use the OpenWeatherMap tool to fetch the current weather information for London, UK.
Action: OpenWeatherMap
Action Input: London, UK
Observation: In London, UK, the current weather is as follows:
Detailed status: broken clouds
Wind speed: 4.12 m/s, direction: 250°
Humidity: 71%
Temperature: 
  - Current: 21.34°C
  - High: 23.68°C
  - Low: 19.36°C
  - Feels like: 21.38°C
Rain: {}
Heat index: None
Cloud cover: 75%
Thought:I now know the final answer
Final Answer: The weather in London, UK right now is broken clouds with a temperature of 21.34°C.

> Finished chain.


'The weather in London, UK right now is broken clouds with a temperature of 21.34°C.'

In [35]:
# Input for Serp API

query = ""

query = input("Question")

query

'What is Claude 2.0?'

In [36]:
agent.run(query)



> Entering new AgentExecutor chain...
I'm not sure what Claude 2.0 is. I should search for more information.
Action: Search
Action Input: "Claude 2.0"
Observation: Claude 2 has improved performance, longer responses, and can be accessed via API as well as a new public-facing beta website, claude.ai. We have ...
Thought:I now know what Claude 2.0 is.
Final Answer: Claude 2.0 is an improved version of Claude, with better performance, longer responses, and the ability to be accessed via API. It also has a new public-facing beta website called claude.ai.

> Finished chain.


'Claude 2.0 is an improved version of Claude, with better performance, longer responses, and the ability to be accessed via API. It also has a new public-facing beta website called claude.ai.'

In [ ]:
# Things to explore:
# 1. How would an agent handle a query that would require multiple tools?
# 2. How to make an agent remember/consider previous responses and queries?
# 3. I should try exploring the other search methods in Google Places API.
# 4. Haven't tried using promptTemplates yet.